import libraries

In [89]:
import pandas as pd
import numpy as np
from pathlib import Path

# DEFINE PATHS

raw_path = Path(r"C:\Users\nyaga\Downloads\merged_datasetssss.csv")

# EXTRACT: LOAD DATA
df = pd.read_csv(raw_path)
print("Shape:", df.shape)
print("Preview:", df.head())

Shape: (14003, 16)
Preview:    StudyHours  Attendance  Resources  Extracurricular  Motivation  Internet  \
0          19          64          1                0           0         1   
1          19          64          1                0           0         1   
2          19          64          1                0           0         1   
3          19          64          1                1           0         1   
4          19          64          1                1           0         1   

   Gender  Age  LearningStyle  OnlineCourses  Discussions  \
0       0   19              2              8            1   
1       0   23              3             16            0   
2       0   28              1             19            0   
3       0   19              2              8            1   
4       0   23              3             16            0   

   AssignmentCompletion  ExamScore  EduTech  StressLevel  FinalGrade  
0                    59         40        0            1   

INITIAL DATA CLEANING 

we perform basic cleaning operations to ensure our dataset is ready for transformation

Tasks include:

  -Checking data types and non-null counts

  -Handling missing values (via imputation or sensible defaults)

   -Removing duplicate rows

These steps prevent inconsistencies during later analysis.

In [90]:
# Basic info
print("Info:", df.info())

# Drop duplicates if any
df = df.drop_duplicates()

# Handle missing values
# Detect and impute or remove missing data appropriately
fill_values={
    "Gender":0, # 0 = female
    "Motivation":1, # 1 = moderate motivation
    "Internet":1, # 1 = yes internet
}
for col, val in fill_values.items():
    if col in df.columns:
        df[col] = df[col].astype('object').fillna(val)
# Verify
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14003 entries, 0 to 14002
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   StudyHours            14003 non-null  int64
 1   Attendance            14003 non-null  int64
 2   Resources             14003 non-null  int64
 3   Extracurricular       14003 non-null  int64
 4   Motivation            14003 non-null  int64
 5   Internet              14003 non-null  int64
 6   Gender                14003 non-null  int64
 7   Age                   14003 non-null  int64
 8   LearningStyle         14003 non-null  int64
 9   OnlineCourses         14003 non-null  int64
 10  Discussions           14003 non-null  int64
 11  AssignmentCompletion  14003 non-null  int64
 12  ExamScore             14003 non-null  int64
 13  EduTech               14003 non-null  int64
 14  StressLevel           14003 non-null  int64
 15  FinalGrade            14003 non-null  int64
dtypes: i

C:\Users\nyaga\AppData\Local\Temp\ipykernel_5480\1743214522.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype('object').fillna(val)
C:\Users\nyaga\AppData\Local\Temp\ipykernel_5480\1743214522.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype('object').fillna(val)
C:\Users\nyaga\AppData\Local\Temp\ipykernel_5480\1743214522.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. T

StudyHours              0
Attendance              0
Resources               0
Extracurricular         0
Motivation              0
Internet                0
Gender                  0
Age                     0
LearningStyle           0
OnlineCourses           0
Discussions             0
AssignmentCompletion    0
ExamScore               0
EduTech                 0
StressLevel             0
FinalGrade              0
dtype: int64

RENAME & MAP ENCODED COLUMNS TO HUMAN-READABLE LABELS

Many categorical variables in the dataset use numeric codes (e.g., 1, 2, 0) to represent categories.

We map these to meaningful names from the Zenodo website (e.g., 1 → Male, 0 → Female) using dictionaries for clarity and interpretability.(manually)

This improves readability and makes downstream visualizations and models easier to interpret

In [91]:
# Explicit mappings from the original notebook
gender_map = {0: 'Female', 1: 'Male', 2: 'Unknown'}
motivation_map = {0: 'Low', 1: 'Medium', 2: 'High'}
extracurricular_map = {0: 'No', 1: 'Yes'}
internet_map = {0: 'No', 1: 'Yes'}
resources_map = {0: 'Low', 1: 'Medium', 2: 'High'}
finalgrade_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

# Apply mappings safely
for col, mapping in {
    "Gender": gender_map,
    "Motivation": motivation_map,
    "Extracurricular": extracurricular_map,
    "Internet": internet_map,
    "Resources": resources_map,
    "FinalGrade": finalgrade_map
}.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)

# Confirm mapping
df.head()


,StudyHours,Attendance,Resources,Extracurricular,Motivation,Internet,Gender,Age,LearningStyle,OnlineCourses,Discussions,AssignmentCompletion,ExamScore,EduTech,StressLevel,FinalGrade
0,19,64,Medium,No,Low,Yes,Female,19,2,8,1,59,40,0,1,D
1,19,64,Medium,No,Low,Yes,Female,23,3,16,0,90,66,0,1,C
2,19,64,Medium,No,Low,Yes,Female,28,1,19,0,67,99,1,1,A
3,19,64,Medium,Yes,Low,Yes,Female,19,2,8,1,59,40,0,1,D
4,19,64,Medium,Yes,Low,Yes,Female,23,3,16,0,90,66,0,1,C


DEFINING THE CORRECT DATA TYPES

In [92]:
# Convert numeric continuous columns
df["StudyHours"] = df["StudyHours"].astype(float)
df["Attendance"] = df["Attendance"].astype(float)
df["Age"] = df["Age"].astype(float)
df["OnlineCourses"] = df["OnlineCourses"].astype(float)
df["AssignmentCompletion"] = df["AssignmentCompletion"].astype(float)
df["ExamScore"] = df["ExamScore"].astype(float)

# Convert categorical / ordinal columns
df["Resources"] = df["Resources"].astype("category")
df["Extracurricular"] = df["Extracurricular"].astype("category")
df["Motivation"] = df["Motivation"].astype("category")
df["Internet"] = df["Internet"].astype("category")
df["Gender"] = df["Gender"].astype("category")
df["LearningStyle"] = df["LearningStyle"].astype("category")
df["Discussions"] = df["Discussions"].astype("category")
df["EduTech"] = df["EduTech"].astype("category")
df["StressLevel"] = df["StressLevel"].astype("category")
df["FinalGrade"] = df["FinalGrade"].astype("category")

# Verify updated data types
df.dtypes

StudyHours               float64
Attendance               float64
Resources               category
Extracurricular         category
Motivation              category
Internet                category
Gender                  category
Age                      float64
LearningStyle           category
OnlineCourses            float64
Discussions             category
AssignmentCompletion     float64
ExamScore                float64
EduTech                 category
StressLevel             category
FinalGrade              category
dtype: object

ADD DERIVED FIELDS

In [93]:
# Example logical derivations
if all(col in df.columns for col in ["Extracurricular", "EduTech"]):
    df["ActiveLearner"] = np.where((df["Extracurricular"] == "Yes") | (df["EduTech"] == "Yes"),
                                   "Yes", "No")

if "FinalGrade" in df.columns:
    df["PassStatus"] = np.where(df["FinalGrade"].isin(["A", "B", "C"]), "Pass", "Fail")


In [98]:
print("✅ Final Transformed Dataset Summary:\n")
(df.info())

# Preview first 10 rows
print(df.head(10))
transformed_path = Path(r"C:\Users\nyaga\OneDrive\Desktop\Semester 2.3\Data Warehousing & Mining\transformed_dataset.csv")
df.to_csv(transformed_path, index=False)
print(f"✅ File saved to {transformed_path}")



✅ Final Transformed Dataset Summary:

<class 'pandas.core.frame.DataFrame'>
Index: 12469 entries, 0 to 14002
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   StudyHours            12469 non-null  float64 
 1   Attendance            12469 non-null  float64 
 2   Resources             12469 non-null  category
 3   Extracurricular       12469 non-null  category
 4   Motivation            12469 non-null  category
 5   Internet              12469 non-null  category
 6   Gender                12469 non-null  category
 7   Age                   12469 non-null  float64 
 8   LearningStyle         12469 non-null  category
 9   OnlineCourses         12469 non-null  float64 
 10  Discussions           12469 non-null  category
 11  AssignmentCompletion  12469 non-null  float64 
 12  ExamScore             12469 non-null  float64 
 13  EduTech               12469 non-null  category
 14  StressLevel          

## Section by Kendi- Advanced data cleaning, Outlier Handling & Standardization 
This section extends the data transformation by:
- Identifying and handling outliers in numeric variables
- Reducing noise in categorical columns
- Standardizing formats for consistency
- Preparing the dataset for week 3 exploratory analysis

In [ ]:
# Detect and handle outliers in key numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    before=df.shape[0]
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    after=df.shape[0]
    print(f"Outliers removed from {col}: {before - after}")

Outliers removed from StudyHours: 45
Outliers removed from Attendance: 0
Outliers removed from Age: 0
Outliers removed from OnlineCourses: 0
Outliers removed from AssignmentCompletion: 0
Outliers removed from ExamScore: 0


Outliers were identified using the Interquartile Range (IQR) method and removed for all numeric features to improve data reliability

In [ ]:
# Identify categorical columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical columns:", cat_cols)

# Step 1: Strip spaces and unify casing
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

# Step 2: Replace known noisy placeholders
noise_values = ["?", "N/A", "None", "Nan", "--", "0", "Null"]

for col in cat_cols:
    df[col] = df[col].replace(noise_values, pd.NA)

# Step 3: Fix obvious typos (optional based on your dataset)
if "Gender" in df.columns:
    df["Gender"] = df["Gender"].replace({
        "Femal": "Female",
        "Femae": "Female",
        "Mle": "Male"
    })

if "Internet" in df.columns:
    df["Internet"] = df["Internet"].replace({
        "Ye": "Yes",
        "Yess": "Yes",
        "N0": "No",
        "Non": "No"
    })

if "Motivation" in df.columns:
    df["Motivation"] = df["Motivation"].replace({
        "Hgh": "High",
        "Lw": "Low",
        "Med": "Medium"
    })

print("✅ Noise reduction complete.")


Categorical columns: ['ActiveLearner', 'Discussions', 'EduTech', 'Extracurricular', 'FinalGrade', 'Gender', 'Internet', 'LearningStyle', 'Motivation', 'PassStatus', 'Resources', 'StressLevel']
✅ Noise reduction complete.


Noise reduction was performed on categorical variables by:
- Trimming whitespace and standardizing text casing.  
- Replacing noisy placeholders (e.g., `"?"`, `"None"`, `"--"`) with `NaN` for reprocessing.  
- Correcting obvious typos in key columns such as `Gender`, `Internet`, and `Motivation`.  
This ensures consistency across all string-based fields for accurate modeling.


In [ ]:
# Round numeric columns to 2 decimal places for consistency
df[numeric_cols] = df[numeric_cols].round(2)

# Reorder columns for better readability
ordered_cols = sorted(df.columns)
df = df[ordered_cols]

print("Dataset standardized successfully.")

Dataset standardized successfully.


All numeric columns were rounded to two decimal places for precision consistency.

Columns were reordered alphabetically to simplify readability

In [ ]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['ActiveLearner', 'Discussions', 'EduTech', 'Extracurricular', 'FinalGrade', 'Gender', 'Internet', 'LearningStyle', 'Motivation', 'PassStatus', 'Resources', 'StressLevel']
Numerical Columns: ['Age', 'AssignmentCompletion', 'Attendance', 'ExamScore', 'OnlineCourses', 'StudyHours']


Categorical and numerical columns were separated to support correlation analysis and modelling in Week 3

In [99]:
from pathlib import Path

output_path =Path(r"C:\Users\nyaga\OneDrive\Desktop\Semester 2.3\Data Warehousing & Mining\final_dataset.csv")
#save final dataset
df.to_csv(output_path, index=False)

print("Final dataset saved successfully at:", output_path)
print("Shape:", df.shape)

Final dataset saved successfully at: C:\Users\nyaga\OneDrive\Desktop\Semester 2.3\Data Warehousing & Mining\final_dataset.csv
Shape: (12469, 18)
